In [2]:
# import necessary imports
import numpy as np
#from numpy_ml.neural_nets.optimizers import Adam
import torch
from torch import nn, optim

In [3]:
# # data preprocessing 
#     # load encoded train/validation/test sets
#     # chunken und batchen
# 
# batch_size = 1
# chunk_size = 1
# 
# def get_batch(input, batch_size):
#     return np.array([[1,2,3], [3,4,7]]), np.array([[2,3,3], [4,7,7]]) # pls adapt. obv. :))))))

In [9]:
def get_batch(input, batch_size,chunk_size):

    input_batch = []
    print(type(input_batch))
    target_batch = []
    idx = np.random.randint(0,len(input)-(chunk_size+1),size=batch_size)
    for i in range(0,len(idx)-1):
        input_batch.append(input[idx[i]:idx[i]+chunk_size])
        target_batch.append(input[idx[i]+1:idx[i]+(chunk_size+1)])
    
    input_batch = np.array(input_batch)
    target_batch = np.array(target_batch)

    return input_batch, target_batch

In [10]:
with open (r"indices_text.txt", 'r') as f:
  indices_text = eval(f.read())
  
x,y = get_batch(indices_text,4,8)
print(x.shape)
print(x)
print(y.shape)
print(y)

<class 'list'>
(3, 8)
[[1263  791  190  360  608  127 1263 1285]
 [ 438 1170 1539 1459  462  622 1401  966]
 [1285  335 1540 1336   14 1285  485 1274]]
(3, 8)
[[ 791  190  360  608  127 1263 1285  413]
 [1170 1539 1459  462  622 1401  966  797]
 [ 335 1540 1336   14 1285  485 1274 1386]]


In [4]:
class neural_embedding:
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.token_embedding_table = np.random.rand(vocab_size, vocab_size)

    def calculate_softmax(self, x):
        """Takes input array x and returns softmax."""
        soft_x = np.exp(x - np.max(x))
        softer_x = soft_x / np.sum(soft_x)
        return softer_x

    def calculate_cross_entropy(self, y_hatless, y_hat):
        """
        Takes target (y_hatless) and prediction (y_hat) and computes cross entropy loss.
        """
        # get vocab_size
        _, _, vocab_size = y_hat.shape        
        y_hat = y_hat.reshape(y_hat.shape[0]*y_hat.shape[1], y_hat.shape[2])
        y_hatless_flat = y_hatless.reshape(-1)
        # one-hot encode targets
        y_hatless_hot = np.eye(vocab_size)[y_hatless_flat]
       
        y_hat = self.calculate_softmax(y_hat)
    
        return -np.sum(y_hatless_hot*np.log(y_hat))
    
    def forward(self, idx, targets=None):
        """
        Implements forward pass with an unnecessary logitte function 
        which i only did not delete because now I'm emotionally attached.
        Args:
            idx(np.array): (B,T) numpy array of integers
            targets(np.array): (B,T) numpy array of integers
        Returns:
            input_logits(np.array)
            sometimes also: targets(np.array)
        """
        batch_size, chunk_size = idx.shape
        logits = np.zeros((batch_size, chunk_size, (self.token_embedding_table[0].size)))

        def logitte(batch_size, chunk_size, input):
            for batch in range(batch_size):
                for chunk in range(chunk_size):
                    # (B,T,C) b=batch_size, t="time"=chunk_size, c=vocab_size
                    logits[batch][chunk] = self.token_embedding_table[input[batch][chunk]]
                    
            return logits

        input_logits = logitte(batch_size, chunk_size, idx)
        
        if targets is not None:
            loss = self.calculate_cross_entropy(targets, input_logits)

            return input_logits, loss

        return input_logits

    
    def backward(self, targets, input_logits):
        # need to do the same reshaping as we did for cross entropy, apparently
        targets_flat = targets.reshape(-1)
        one_hot_targets = np.eye(self.vocab_size)[targets_flat]

        # shape after: ((batch_size*chunk_size), vocab_size)
        input_logits_2d = input_logits.reshape(input_logits.shape[0]*input_logits.shape[1], input_logits.shape[2])

        # somehow this is supposedly the combiantion of the derivative of softmax with the derivative of the CCE
        delta = one_hot_targets - input_logits_2d

        # want shape (80,80) for matrix multiplication, but with correct indices (use one-hot targets for that)
        delta_indexed = np.dot(one_hot_targets.transpose(),delta)

        # compute gradient for weight matrix: dot product between the transpose of the to layer and delta vector computed above
        gradient = (self.token_embedding_table @ delta_indexed) 

    
        return gradient

    

In [11]:
# currently do not use loss at all, so something is probably very wrong

def train(model, text, optimiser, token_embedding_table):
    batch_size=32
    chunk_size = 8
    for steps in range(10): # TODO: please increase
        # sample batch of data
        xb, yb = get_batch(text, batch_size, chunk_size) # TODO: pls adapt to above fct
        optimiser.zero_grad()  # reset gradients
        # get logits and loss
        logits, loss = model.forward(xb, yb)
        print(f"loss: {loss}")

        gradient = model.backward(yb, logits)
        token_embedding_table.grad = torch.tensor(gradient, dtype=torch.float32)
        optimiser.step()  # apply gradients to parameters
        diff = model.token_embedding_table - np.array(token_embedding_table)
        print(f"Difference in weights before - afte Adam: {diff}")
        model.token_embedding_table = np.array(token_embedding_table)
        

In [13]:
with open (r"indices_text.txt", 'r') as f:
    indices_text = eval(f.read())
with open(r"vocab_train.txt", 'r') as f:
    vocab_train = eval(f.read())

In [14]:
vocab_size = len(vocab_train)

my_neural_embedding = neural_embedding(vocab_size)
param_dict = {"weight": my_neural_embedding.token_embedding_table} # one entry, key is weight and value is my_neural

tensor = torch.tensor(my_neural_embedding.token_embedding_table, dtype=torch.float32)
optimiser = optim.Adam([tensor], lr=0.3)
train(my_neural_embedding, indices_text, optimiser, tensor)


<class 'list'>
loss: 3202.6472473793574
Difference in weights before - afte Adam: [[-0.29999998 -0.29999998 -0.29999997 ... -0.29999999 -0.29999999
  -0.29999995]
 [-0.29999997 -0.30000003 -0.29999998 ... -0.30000001 -0.29999997
  -0.29999998]
 [-0.29999997 -0.29999999 -0.29999998 ... -0.29999993 -0.3
  -0.30000001]
 ...
 [-0.29999999 -0.30000001 -0.29999993 ... -0.29999996 -0.29999994
  -0.29999997]
 [-0.29999998 -0.29999987 -0.29999993 ... -0.29999996 -0.29999999
  -0.29999995]
 [-0.30000003 -0.30000001 -0.3        ... -0.29999998 -0.29999995
  -0.30000001]]
<class 'list'>
loss: 3197.2258681386966
Difference in weights before - afte Adam: [[-0.28314358 -0.27951425 -0.2813983  ... -0.28145868 -0.28104
  -0.27603263]
 [-0.2811556  -0.27822626 -0.27827388 ... -0.2795955  -0.27734727
  -0.27823392]
 [-0.28287196 -0.2783522  -0.27953547 ... -0.27827358 -0.27961063
  -0.27764198]
 ...
 [-0.2839327  -0.281659   -0.28297532 ... -0.28117943 -0.28143716
  -0.2798254 ]
 [-0.2848431  -0.28276253

IndexError: index 1541 is out of bounds for axis 0 with size 1541